<font size="5">
 <div class="alert alert-block alert-info"><b>Master in Data Science - Iscte <b>
     </div>
</font> 
 
 
     
    
  <font size="5"> OEOD </font>
  
  
  
  <font size="3"> **Diana Aldea Mendes**, November 2024 </font>
  
   
  <font size="3"> *diana.mendes@iscte-iul.pt* </font> 
  
    
 
  
    
  <font color='blue'><font size="5"> <b>Week 4 - Q-learning<b></font></font>

_________________

# Q-learning algorithm

- Implementation of the Q-learning algorithm for a grid-world context
- Gridworld problem involves an agent navigating a grid to reach a goal while avoiding obstacles

## Environment

- We define a 5x5 grid with the following properties:

    - The agent starts at the top-left corner (0, 0).
    - The goal is at the bottom-right corner (4, 4) with a high positive reward of +10.
    - Obstacles, with negative rewards (-1), are placed at grid points (1, 1) and (2, 3).
    - The rest of the grid is empty and can either have neutral (0) or small negative rewards (e.g., -0.1, but in this post we consider only the neutral case), encouraging the agent to reach the goal efficiently.

In [ ]:
### define the grid

import numpy as np

grid = np.zeros((5, 5))
grid[-1, -1] = 10  # Goal with a high positive reward
grid[1, 1] = -1  # Obstacle with a negative reward
grid[2, 3] = -1  # Another obstacle
print(grid)


### reward of +10 when reaching the goal, -1 for hitting obstacles, and 0 for navigating other empty cells.

## Q-learning

- The essence of Q-Learning lies in the Q-table, a data structure that stores the estimated values (Q-values) of taking different actions in different states.These Q-values represent the expected future rewards for each action-state pair. 
- The algorithm follows these key steps:
1. Initialization: The Q-table is initialized with arbitrary values, often zeros.
2. Exploration and Exploitation: The agent interacts with the environment,taking actions based on the Q-table. It balances exploration (trying random actions to gather information) with exploitation (choosing actions with the highest Q-values to maximize rewards).
3. Action Selection: The agent selects an action based on the Q-values in the current state. It might choose the action with the highest Q-value (greedy approach) or introduce some randomness for exploration (epsilon-greedy approach).
4. Observation and Reward: The agent observes the next state and receives a reward based on the action taken.
5. Q-Value Update: The Q-value for the previous state-action pair is updated using the Bellman equation. The equation combines the immediate reward with the discounted maximum expected future reward from thenext state.
6. Iteration: Steps 2–5 are repeated until the Q-table converges, meaning theQ-values stabilize, indicating the agent has learned the optimal policy.

- Write the Q-learning formula as:

$$
Q(s, a)=(1-\alpha) Q(s, a)+\alpha\left(R+\gamma \max _{a^{\prime}} Q\left(s^{\prime}, a^{\prime}\right)\right)
$$

- It can be interpreted as:

    - $(1 — α) Q(s, a)$: The agent’s expected reward based on old knowledge.
    - $α R$: The new reward based on recent exploration.
    - $γ max Q(s’, a’)$: The maximum expected future reward, weighted by $γ$, the discount factor.

- This balance between old knowledge, new experience, and future reward drives the agent to learn the best policy over time.


_____________________

- Explore vs. Exploit
    - If we let the agent always randomly choose an action, it could eventually learn the Q-table, but the process will never be efficient. On the contrary, if we only choose an action based on the maximization of the Q-value, the agent will tend to always take the same route, overfitting the current environment setup. Furthermore it will suffer from great variance as it will not be able to find the proper route in another environment setup.

    - To prevent those two scenarios to occur and to try to find a trade-off, we add another hyperparameter epsilon ϵ, which is the probability with which we choose a random action instead of the one computed from the Q-table. Playing with this parameter allows us to find an equilibrium.

In [ ]:
import numpy as np

# Parameters
grid_size = 5
gamma = 0.1  # Discount factor
alpha = 0.1  # Learning rate
epsilon = 0.1  # Exploration rate
num_episodes = 1000

grid = np.zeros((5, 5))
grid[-1, -1] = 10  # Goal with a high positive reward
grid[1, 1] = -1  # Obstacle with a negative reward
grid[2, 3] = -1  # Another obstacle

# Action space: up, down, left, right
actions = ['up', 'down', 'left', 'right']
q_table = np.zeros((grid_size, grid_size, len(actions)))

# Define the transition function
def get_next_state(state, action):
    i, j = state
    if action == 'up' and i > 0:
        return (i - 1, j)
    elif action == 'down' and i < grid_size - 1:
        return (i + 1, j)
    elif action == 'left' and j > 0:
        return (i, j - 1)
    elif action == 'right' and j < grid_size - 1:
        return (i, j + 1)
    return state

# Define the reward function
def get_reward(state):
    return grid[state]

# Epsilon-greedy action selection
def choose_action(state):
    if np.random.rand() < epsilon:
        return np.random.choice(actions)
    else:
        return actions[np.argmax(q_table[state[0], state[1], :])]

# Q-learning algorithm
for episode in range(num_episodes):
    state = (0, 0)  # Start at top-left corner
    done = False
    while not done:
        action = choose_action(state)
        next_state = get_next_state(state, action)
        reward = get_reward(next_state)
        action_index = actions.index(action)

        # Update Q-table using Q-learning update rule
        q_table[state[0], state[1], action_index] += alpha * (reward + gamma * np.max(q_table[next_state[0], next_state[1], :]) - q_table[state[0], state[1], action_index])
        
        state = next_state

        # End episode if we reach the goal
        if reward == 10:
            done = True

# Display learned Q-values
print(q_table)

_____________________


# Taxi Problem - Gym

- Solve Taxi Problem by using Q-learning Algorithm
- Taxi problem: an agent must navigate a grid to transport passengers to their destinations.
- Grid problem, grid size = 5x5
- The environment has several fixed locations (denoted by the letters R, Y, G, and B for Red, Yellow, Green, and Blue). 
- These letters represent the possible pick-up and drop-off locations for passengers.
- The taxi starts in a random position, and at each episode, the passenger’s location and destination are randomly assigned to one of these four spots. 
- The agent (taxi) must figure out the best sequence of moves to:

    - Navigate to the passenger’s location,
    - Pick up the passenger,
    - Navigate to the dropoff location,
    - Drop off the passenger successfully.
    
- The goal of the agent is to maximize the reward by performing the correct sequence of actions in the minimum number of steps.

_________________

- Two main steps in the process
1. Training the agent using Q-learning
2. Testing the trained agent by visualizing its behavior

In [ ]:
### import libraries

import gymnasium
import numpy as np
import random
from time import sleep
from IPython.display import clear_output

ModuleNotFoundError: No module named 'gym'

## Step 1: Q-learning

- The Environment: We initialize the Taxi-v3 environment, which provides a grid with fixed pickup and dropoff points.
- Q-Table Initialization: We initialize a Q-table (a 2D array) with dimensions corresponding to the number of states and actions in the environment. This table will store the learned action values for each state.
- Hyperparameters: learning rate (learning_rate), discount rate (discount_rate), and exploration-exploitation tradeoff (epsilon) 
- Episodes and Steps: The training runs over a set number of episodes, where each episode represents an attempt by the agent to solve the taxi problem from a new random initial state.
- Epsilon Decay: Epsilon governs the agent’s exploration-exploitation balance. Over time, we decay epsilon using an inverse decay rule to slowly shift from exploring to exploiting the learned strategy as training progresses.

____________________

- The Q-learning algorithm iteratively updates the Q-table based on the rewards received after each action. Over time, the agent learns which actions yield the best rewards in different states.



In [ ]:
# create Taxi environment
env = gym.make('Taxi-v3', render_mode='ansi')


NameError: name 'gym' is not defined

In [ ]:
# initialize q-table
state_size = env.observation_space.n
action_size = env.action_space.n
qtable = np.zeros((state_size, action_size))

In [ ]:
# hyperparameters
learning_rate = 0.9
discount_rate = 0.8
epsilon = 0.2
decay_rate = 0.005

# training variables
num_episodes = 2000
max_steps = 99 # per episode



In [ ]:


print("Training the agent...")

for episode in range(num_episodes):
    # Reset the environment
    state = env.reset()[0]
    step = 0
    done = False

    for step in range(max_steps):
        # Exploration-exploitation tradeoff
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(qtable[state,:])  # Exploit

        # Take an action and observe the reward
        output = env.step(action)
        new_state, reward, done, info = output[0], output[1], output[2], output[3]

        # Q-learning update
        qtable[state,action] += learning_rate * (reward + discount_rate * np.max(qtable[new_state,:]) - qtable[state,action])

        # Update state
        state = new_state

        if done:
            break

    # Decay epsilon to reduce exploration over time
    epsilon = 1.0 / (1.0 + decay_rate * episode)

print(f"Training completed after {num_episodes} episodes.")

## Testing step
- When training is complete, visualize the agent’s behavior by letting it solve a few episodes of the Taxi problem.

In [1]:
def visualize_agent(env, qtable, episodes=5, max_steps=100):
    for episode in range(episodes):
        state = env.reset()[0]
        done = False
        print(f"Episode {episode + 1}\n")
        sleep(1)

        for step in range(max_steps):
            clear_output(wait=True)
            print(env.render())
            sleep(0.5)  # Adjust speed of animation

            # Choose action based on Q-table
            action = np.argmax(qtable[state, :])
            output = env.step(action)
            new_state, reward, done, info = output[0], output[1], output[2], output[3]            
            
            state = new_state

            if done:
                print(f"Episode finished after {step + 1} timesteps\n")
                sleep(2)
                clear_output(wait=True)
                break

# Visualize the trained agent
visualize_agent(env, qtable, episodes=5)

NameError: name 'env' is not defined